In [ ]:
## Loading Config Files

import os 
import json
import sys

sys.path.append(os.path.dirname(os.getcwd()))
with open('../config/runner_config.json') as json_file:
    json_data = json.load(json_file)


In [ ]:
## Data Aggreagator Run
from deployment.Data_Retreiver import Data_Retreiver
from deployment.Data_Aggregator import Aggregator

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

sql_table = json_data["sql_aggregate"]
sql_table_raw = json_data["sql_raw_energy"]


# Create Threads for All the devices
aggre = Aggregator(devices,sql_table,sql_table_raw,sql_addr,sql_port, sql_user, sql_pw, sql_db)

#Do step and get the final output that would be printed to the log
aggre.do_step()
print(aggre.getLatest())

In [ ]:
## Conctrol Enactor 
from deployment.Control_Enactor import Enactor


devices = json_data["devices"]

sftp_location = json_data["sftp_location"]
sftp_port = json_data["sftp_port"]
sftp_username = json_data["sftp_username"]
sftp_password = json_data["sftp_password"]
sftp_key_location = json_data["sftp_key_location"]
sftp_directory = json_data["sftp_directory"]

#Global for Unit Test
enact = Enactor(devices, sftp_location, sftp_port, sftp_username, sftp_password, sftp_key_location, sftp_directory)

# Run Examples
print("Retreive - Set Points")
print(enact.retreive_previous_plan_DC(list(devices.keys())[1]))
print(enact.retreive_previous_plan_AC(list(devices.keys())[9]))
print("Enact Set points")
print(enact.enact_light_plan(list(devices.keys())[1], 2100, 2100))
print(enact.enact_socket_plan(list(devices.keys())[9], 1000000))
print("Enact Same set points")
print(enact.enact_light_plan(list(devices.keys())[1], 2100, 2100))
print(enact.enact_socket_plan(list(devices.keys())[9], 1000000))
print("Enact Diffrent set points")
print(enact.enact_light_plan(list(devices.keys())[1], 4320, 4320))
print(enact.enact_socket_plan(list(devices.keys())[9], 1200000))


In [ ]:
#Data retreiver Run
import datetime
import pandas as pd 
from deployment.Data_Retreiver import Data_Retreiver
devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

print("--Retreive filled data--\n",data.get_unsent_logs(datetime.datetime.now()))
print("--Retreive filled data--\n",data.retreive_filled_aggre(datetime.datetime.now() - datetime.timedelta(days=20), 2))

dev_light = "Nursery_1A_CPE_No_2"
dev_socket = "Playground_AC_socket_No_1"
group = ["Nursery_1A_CPE_No_1", "Nursery_1A_CPE_No_2", "Nursery_1B_CPE_No_3", "Nursery_1B_CPE_No_4",
         "Nursery_1C_CPE_No_5", "Nursery_1C_CPE_No_6"]
hour = 6

pd.set_option('display.max_columns', 30)
dt_ts = datetime.datetime.now()
print("--Forecast--\n", data.retreive_latest_forecast(dt_ts).head(5))
print("--Raw Sys--\n", data.retreive_latest_raw_system_snapshot(dt_ts).head(5))
print("--Priority--\n", data.retreive_latest_priority(dt_ts).head(5))
print("--Group Energy--\n", data.get_total_energy_for_group(group, dt_ts))
print("--Aggregated Values --\n", data.retreive_aggregared_values(dt_ts))
print("--AVG for Hour Values --\n", data.retreive_average_vals_for_hour(dt_ts, hour,8))
print("--Average P Light--\n", data.retreive_average_P_lights(dev_light, dt_ts))
print("--AC Session--\n", data.retreive_AC_Session(dev_socket, dt_ts))
print("--AC Energy--\n", data.retreive_AC_Energy(dev_socket, dt_ts))
print("--Light Session--\n", data.retreive_Light_Session(dev_light, dt_ts))
print("--Light Energy--\n", data.retreive_Light_Energy(dev_light, dt_ts))
print("--Quota Latest Light--\n", data.get_latest_quota(dev_light, dt_ts))
print("--Quota Latest Socket --\n", data.get_latest_quota(dev_socket, dt_ts))



In [ ]:
from deployment.Battery import Battery

bat = Battery(state_of_charge=94.0, battery_capacity=18.79, max_discharge=40.0, charge_efficiency=0.85)
print(bat)
print("---Storage Capacity---\n", bat.get_storage_capacity_left())
print("---Discharge Capacity---\n", bat.get_discharge_capacity_left())
print("---Discharge Battery---\n", bat.discharge_battery(1.9))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(0.4))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)


In [ ]:
# Controller Run

#Data retreiver Run
import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Control_Enactor import Enactor
from deployment.Controller import Controller

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

## Conctrol Enactor 
sftp_location = json_data["sftp_location"]
sftp_port = json_data["sftp_port"]
sftp_username = json_data["sftp_username"]
sftp_password = json_data["sftp_password"]
sftp_key_location = json_data["sftp_key_location"]
sftp_directory = json_data["sftp_directory"]

#Global for Unit Test
enact = Enactor(devices, sftp_location, sftp_port, sftp_username, sftp_password, sftp_key_location, sftp_directory)

#Controller Init
ts_step = datetime.datetime.now()  # - datetime.timedelta(hours=3)

allocation = json_data["allocation"]
if 'System_Data' in allocation:
    del allocation['System_Data']

control = Controller(data, enact, allocation, 4)
print(control.getLatest())

control.do_step(ts_step)
print(control.getLatest())

In [ ]:
## Forecaster V1 - Simple not ML

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Battery import Battery
from deployment.Forecaster import Forecaster

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)
forecast = Forecaster(Battery, data, devices)


curr_ts = datetime.datetime.now()
print("Running Forecasting -- ", curr_ts, " --")
forecast_period = 48
forecast.do_step(forecast_period, curr_ts)
print(forecast.getLatest())
#Latest Values
display(forecast.latest_forecast)

In [ ]:
##Forecaster V2 - ML and PV Models
## Forecaster V1 - Simple not ML

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Battery import Battery
from deployment.Forecaster_v2 import Forecaster

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)
forecast = Forecaster(Battery, data, devices)


curr_ts = datetime.datetime.now()
print("Running Forecasting -- ", curr_ts, " --")
forecast_period = 48
forecast.do_step(forecast_period, curr_ts)
print(forecast.getLatest())
#Latest Values
display(forecast.full_df)

In [ ]:
##Remote Logger Example

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Remote_Logger import RemoteLogger

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

mqtt_address = json_data["mqtt_address"]
mqtt_port = json_data["mqtt_port"]
mqtt_username = json_data["mqtt_username"]
mqtt_password = json_data["mqtt_password"]

data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

rl = RemoteLogger(data, mqtt_address, mqtt_port, mqtt_username, mqtt_password)

rl.do_step()
print(rl.getLatest())